<a href="https://colab.research.google.com/github/Manisha2297/RestroRecommender/blob/main/NLTKmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
import string
from scipy import sparse

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#mount your own drive to avoid downloading the data multiple time 

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
df_review = pd.read_csv('/content/gdrive/MyDrive/YelpDataset/CsvFiles/review_filtered.csv')
df_review.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# df_review = pd.read_csv('/content/gdrive/MyDrive/YelpDataset/CsvFiles/yelp_academic_dataset_review.csv')
# df_review.drop('Unnamed: 0', axis=1, inplace=True)

In [21]:
restaurant_df = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/restuarant_df')

In [6]:
restaurant_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,404 E Green St,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...",pQeaRpvuhoEqudo3uymHIQ,"Ethnic Food, Food Trucks, Specialty Food, Impo...",Champaign,"{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",1,40.1104,-88.2331,The Empanadas House,61820,5,4.5,IL
1,2831 Parmenter St,"{'BikeParking': 'True', 'Caters': 'True', 'Whe...",M_guz7Dj7hX0evS672wIwA,"Desserts, Food, Ice Cream & Frozen Yogurt",Middleton,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",1,43.1053,-89.5101,Chocolate Shoppe Ice Cream,53562,6,3.5,WI
2,NaN,{'BusinessAcceptsCreditCards': 'True'},mKTq1T_IAplDpHUcMzOXkw,"Hotels & Travel, Transportation, Taxis, Beer, ...",Gastonia,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,35.2528,-81.1527,Tipsy Ryde,28054,3,3.5,NC
3,"15480 Bayview Avenue, unit D0110","{'RestaurantsTableService': 'False', 'Restaura...",eBEfgOPG7pvFhb2wcG9I7w,"Restaurants, Cheesesteaks, Poutineries",Aurora,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",1,44.011,-79.4487,Philthy Phillys,L4G 7J1,4,4.5,ON
4,300 John Street,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",lu7vtrp_bE9PnxWfA8g4Pg,"Japanese, Fast Food, Food Court, Restaurants",Thornhill,NaN,1,43.8205,-79.3985,Banzai Sushi,L3T 5W4,7,4.5,ON


In [22]:
# NC
business_id_list = list(restaurant_df[restaurant_df['state']=='NC']['business_id'].values)

In [ ]:
# df_review_filtered = df_review[df_review['business_id'].isin(business_id_list)]

In [ ]:
# pd.DataFrame(df_review_filtered.user_id.unique(), columns=['user_id']).to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/unique_users_df')

In [ ]:
# df_review_filtered.to_csv('/content/gdrive/MyDrive/YelpDataset/CsvFiles/review_filtered_ON.csv')

In [9]:
df_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,HQl28KMwrEKHqhFrrDqVNQ,0,2015-12-05 03:18:11,0.0,LG2ZaYiOgpr2DK_90pYjNw,5.0,I love Deagan's. I do. I really do. The atmosp...,1.0,V34qejxNsCbcgD8C0HVk-Q
1,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0.0,6TdNDKywdbjoTkizeMce8A,4.0,"Oh happy day, finally have a Canes near my cas...",0.0,UgMW8bLE0QMJDCkQ1Ax5Mg
2,nlxHRv1zXGT0c0K51q3jDg,0,2013-05-07 07:25:25,0.0,L2O_INwlrRuoX05KSjc4eg,5.0,This is definitely my favorite fast food sub s...,2.0,5vD2kmE25YBrbayKhykNxQ
3,Pthe4qk5xh4n-ef-9bvMSg,0,2015-11-05 23:11:05,0.0,ZayJ1zWyWgY9S_TRLT_y9Q,5.0,"Really good place with simple decor, amazing f...",1.0,aq_ZxGHiri48TUXJlpRkCQ
4,e_BiI4ej1CW1F0EyVLr-FQ,0,2015-02-16 06:48:47,0.0,JA-xnyHytKiOIHl_ztnK9Q,5.0,Most delicious authentic Italian I've had in t...,0.0,P6apihD4ASf1vpPxHODxAQ


In [23]:
#Select only stars and text
yelp_data = df_review[['business_id', 'user_id', 'stars', 'text']]

In [24]:
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [25]:
yelp_data['text'].fillna("", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [26]:
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they have any punctuation
    nopunc = [char for char in text if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])

In [ ]:
# size = 0
# processed_text_df = pd.DataFrame()
# for i in range(2000000, len(yelp_data), 10000):
#   processed_text_df = pd.concat([processed_text_df, yelp_data['text'][i:i+10000].apply(text_process)])
#   size+=10000

#   if len(processed_text_df) >= size:
#     if len(processed_text_df)%100000==0:
#       print(len(processed_text_df))
#     processed_text_df.to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df_2')
#     del processed_text_df
#     processed_text_df = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df_2')
  
#   else:
#     print(size)
#     break


In [ ]:
len(yelp_data)

4607291

In [ ]:
# processed_text_df_1 = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df_1')

In [ ]:
# processed_text_df_final = pd.concat([processed_text_df_1, processed_text_df])

In [ ]:
# processed_text_df_final.to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df')

In [27]:
processed_text_df_final = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df')

In [28]:
yelp_data['text'] = processed_text_df_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
yelp_data = yelp_data[yelp_data['business_id'].isin(business_id_list)]

In [30]:
yelp_data.shape

(316884, 4)

In [31]:
yelp_data.to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/yelp_data_df_NC')

In [ ]:
# del processed_text_df_1
del df_review

In [4]:
yelp_data = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/yelp_data_df_NC')

In [23]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [24]:
userid_df.head()

,user_id,text
16,XivuRnPq3D-K5cPiBTztIA,good want several times hoping experience alwa...
41,iAQFEArbXp1WUrseOFTLMA,GREAT SERVICE food amazing reasonably priced O...
47,oAFF_A02Ws0mwH-lkbzyMQ,Pros Fun atmosphere great people watching mech...
56,AVxQMKJsNvyNPbCJsinvYA,Fast friendly always make sure order correct e...
61,XBRgHoPjOi4GgcYFCvYElQ,3 stars food service awful Appetizers ordered ...


In [28]:
userid_df[userid_df['user_id']=='XBRgHoPjOi4GgcYFCvYElQ']['text']

61         3 stars food service awful Appetizers ordered ...
53089      know married right person wake 15 minutes Coca...
90854      Surprise eating thought stopping pick sweets I...
551356     like frosting Jack Frost Donuts best donuts hi...
1044302    place highly overrated First impressions decen...
1926222    order burger get burger mind mind got small bu...
3755762    really pretty inside service food okay portion...
4293787    Meh Came last night say service 5 stars Wait t...
Name: text, dtype: object

**Join the text with for each user_id and business_id**

In [29]:
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
business_df = business_df.groupby('business_id').agg({'text': ' '.join})

In [30]:
userid_df.loc['XBRgHoPjOi4GgcYFCvYElQ']['text']

'3 stars food service awful Appetizers ordered food brought wondering food Saw multiple tables served sat around waiting check back know married right person wake 15 minutes Coca opens get time belabor point many others brought except say going weekend get open Otherwise guarantee wait bias going bring comfortably seated stage call gotten early able pick spot order sage sausage entrees One get something savory like Bacon Avocado Omelet bring friend right one something sweet like Pumpkin Pancakes happen go fall Sharing caring make sure everyone gets ample opportunity sample spectrum goodness Get requisite coffee freshlyjuiced juice sit back bask sunlight dining rooms spacious feel go walk cause going stuffed need work Surprise eating thought stopping pick sweets Im glad finally check place bookmarks Bikaner exactly close live pretty safe say worth trip Pindi Chana delicious Afghani Pulao even though really dislike dried fruits food Minus one star server kept forgetting drinks Nice guy f

**Creating Tfidf vectors for userid_df and businessid_df with 5000 Features**

In [33]:
# userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])

In [34]:
sparse.save_npz("/content/gdrive/MyDrive/YelpDataset/PickleFiles/userid_vectors_NC.npz", userid_vectors)

In [31]:
userid_vectors = sparse.load_npz("/content/gdrive/MyDrive/YelpDataset/PickleFiles/userid_vectors_NC.npz")

In [35]:
#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])

In [36]:
sparse.save_npz("/content/gdrive/MyDrive/YelpDataset/PickleFiles/businessid_vectors_NC.npz", businessid_vectors)

In [32]:
businessid_vectors = sparse.load_npz("/content/gdrive/MyDrive/YelpDataset/PickleFiles/businessid_vectors_NC.npz")

In [39]:
from tqdm import tqdm

chunk_size = 50000
chunks = [x for x in range(0, yelp_data.shape[0], chunk_size)]

for i in range(0, len(chunks) - 1):
    print(chunks[i], chunks[i + 1] - 1)

0 49999
50000 99999
100000 149999
150000 199999
200000 249999
250000 299999


In [ ]:
userid_rating_matrix = pd.DataFrame()
user_col = 'user_id'
item_col = 'business_id'
rating_col = 'stars'

for i in tqdm(range(0, len(chunks) - 1)):
    chunk_df = yelp_data.iloc[ chunks[i]:chunks[i + 1] - 1]
    interactions = (
    chunk_df.groupby([user_col, item_col])[rating_col]
    .sum()
    .unstack()
    .reset_index()
    .fillna(0)
    .set_index(user_col)
    )
    print (interactions.shape)
    userid_rating_matrix = userid_rating_matrix.append(interactions, sort=False) 

In [8]:
yelp_data.head()

,business_id,user_id,stars,text
16,oyjkEn5fAqcGNsBbqc8LTw,XivuRnPq3D-K5cPiBTztIA,1.0,good want several times hoping experience alwa...
41,gfhWUixZ4MmEAC1DtTsodA,iAQFEArbXp1WUrseOFTLMA,5.0,GREAT SERVICE food amazing reasonably priced O...
47,l_kefVF1frmC0xRW2YkvUA,oAFF_A02Ws0mwH-lkbzyMQ,2.0,Pros Fun atmosphere great people watching mech...
56,srHBs-NlG0upDlambkURbA,AVxQMKJsNvyNPbCJsinvYA,5.0,Fast friendly always make sure order correct e...
61,osSwv6CJy5hDKQdOKeyTow,XBRgHoPjOi4GgcYFCvYElQ,2.0,3 stars food service awful Appetizers ordered ...


In [11]:
yelp_data.sort_values(by=['user_id','business_id'])

,business_id,user_id,stars,text
407517,k-drEjxKmfqllwfY90STfA,--1yLzzEz00g2ajbHn3OHQ,5.0,last restaurant modern visited prior leaving L...
2519540,ADgacmZ-qXrSOhMfU6bmTA,--3WaS23LcIXtxyFULJHTA,3.0,Server awesome Food fast delish Beer cold ask
248467,M0pkmBUi_CI0qrzN7ee80Q,--3WaS23LcIXtxyFULJHTA,5.0,little ago honey waitress Aimee super attentiv...
4146939,zaLFPpm-t89qSDOaFVgwDQ,--3WaS23LcIXtxyFULJHTA,4.0,best breakfast Hendersonpossibly Vegas Fresh i...
3848792,aE-02jgBVhV56x0kbfvSIA,--5-MQZPsn2CJkyMFQkOgQ,5.0,Im one chain restaurants Monday weeknight 10pm...
...,...,...,...,...
3668802,wO9TPd-8UoY7vmSsqgf9qA,zzw0Z6-_VDp9ShIRSKIsQw,4.0,Hands best fine dining Italian Charlotte proba...
2148004,y14TQfHRL5Gkk-CorcR6bg,zzw0Z6-_VDp9ShIRSKIsQw,5.0,favorite place get bahn mi Great ingredient gr...
2681662,zHrvBbRMSV9SFqZC6qa20Q,zzw0Z6-_VDp9ShIRSKIsQw,2.0,barbecue sandwich amazing Blacks favorite rest...
4255443,tMKblwX0CeCDZbCz6ruyPg,zzwYhKPhnZQPushMyzgtDg,2.0,goto casual dinner love menu selection Ive cam...


In [9]:
yelp_data[((yelp_data['business_id']=="-5XuRAfrjEiMN77J4gMQZQ") & (yelp_data['user_id']=="--1yLzzEz00g2ajbHn3OHQ"))]

,business_id,user_id,stars,text


In [5]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])

In [14]:
userid_rating_matrix.T[~userid_rating_matrix.T['fDdaVDFZ6LJVh1-JbrM7DQ'].isna()]['fDdaVDFZ6LJVh1-JbrM7DQ']

business_id
-0QtTRrAMn6DKLZNef3Ojg    1.0
Name: fDdaVDFZ6LJVh1-JbrM7DQ, dtype: float64

In [6]:
userid_rating_matrix.to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/userid_rating_matrix_NC')

In [13]:
userid_rating_matrix.tail()

business_id,--7zmmkVg-IMGaXbuVd0SQ,--KCl2FvVQpvjzmZSPyviA,--cZ6Hhc9F7VkKXxHMVZSQ,-0QtTRrAMn6DKLZNef3Ojg,-2pQf1ceDZyE2ReCNbj-3A,-33_OPx1aKM22qxioPgJ_Q,-3PHn7uGXNaOytVl1JnovQ,-5L8zOxibac-vBrsYtxXbQ,-5XuRAfrjEiMN77J4gMQZQ,-8Oaw94x2pYJCVP8ZtuTTw,-8fOqUWFX_1qiKggIt5LnA,-8iwcXhLnyqbLgvcrJGgaw,-Bb3kfdHIX1qm2RvtHnOTQ,-BplplCOayCupKnRVZcclw,-FpKsMctW7nmFEDeHe4-ag,-GWI_4FktOD7fwHW1DgZCA,-HIwk8B2mWzGS84vuM8ybA,-I27yVw2x_-31_OOMSH02w,-JkxHGA3nBASzwYdBDAiow,-KqwcJ10CUzl-hEwgwGhUA,-L9TeQN_JgLBvqf9wmsBVQ,-LAiusd8GnWFR8atxbT-Vw,-LGIVl0HBoiafmqaNCdTow,-NkvJ2uwK_29GmjeOrY0LQ,-NpRmzPCs3jO-iW9gGkXtA,-OXGCSOIcKgla8yXoHaSwA,-PDte5c2UV23_s9TID-4Kw,-PO8-_JzzUw-vgpyqIq3sQ,-PYiTrmibO-XxqpRRF4EoA,-Pd-IBDWWwRNWzBBFTb7iQ,-QC71Wuh0TyJI0RAZ3d0Iw,-RP7Sw9lqHySxzzRFaeYiQ,-RTEVGUP2wnKB3PV-_fueg,-Rr4TuM1tWhagDcLTp_qUQ,-T0M3exc6tC6sw5LjtVdDQ,-TAQY5daovkW45iYGVfbyA,-U0liwRSHGYBEMYVj8kFmQ,-UNnYuoXnup4k-S1SC0SzA,-UV2tt3gy57-5a29paA_4w,-UxAJfViirpbkbth6_RKTw,...,zOMiDo5iGn5dTrckP4F_Ig,zPUjrM223oSulvRRhpkAmQ,zPfSGU3hzNPNV8twXBiayA,zQydbmG0HmZnrbu6SGiDcw,zRr45TrTgq8XAKwvjM0W_Q,zUHIDqm_UKdnSygmWKtyRg,zW2SAaWtoDNM_JTkIW8Jtg,zX--4nk6LshQ3D79HPACxw,zXEMRlEubK4iJeuYndgIFw,zXsB61ZYYsTCaKYOTYr1Tg,zYDAMT8UjETKRSYGXsbL3A,z_rVJjsUZ8yKvSvPLrGkzA,za1ocYYyi5uKdQ0Q3M3BQQ,zaLFPpm-t89qSDOaFVgwDQ,zb6RpofFD-rmpXqHVdXZbQ,zbN-2RROOpOfaYJh8lnV3Q,zbbQdTc22S9ROibEimKHAw,zcfmbmOEX83YL04f9_Y0BA,zdoiA0CuVX_90K3ccYCdIQ,zdvAsIl9D5ECa2M0u9cQrg,zenqYOv9U2pXb-dwzETIXg,zfPRTgGR9U8T6Rdnos_mAQ,zgtgMlvFk5Ilrs1yj78tdA,ziNGerrnVTaVHQ5kVngfZA,zibinklC3wVDRMhMOsDsoA,zki75FDORLkeFQvf2vmF8w,zlX5HIwcZUIssEgIwYaXLA,zllWgXyzGWhTG00FK0IUtw,zpGCFAt_MydbQN4EsXee6w,zpVxylLC8jnHY1xK1sWpjA,zr7grc6VLxwuo3iXLAs1oA,zr8Ojsm-yNhvfTzBzcKU9w,zrEmWr6ue5yfxnKr4E2_jg,zrxIW7w0zs5JueXIwQaaEQ,zsR57scngUPI5W686GmSGA,zsu7d3Oed4k8ABUChSp-1g,zwxBTPOR020qUXjzPTWBFw,zy-XZ2z_KmnISE1Gl99w1Q,zy1B_cz5Urt7dSjfR5TPvw,zzmIMvqiBJ_-wVKg_OnGpw
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
zzstF6X4o3TaWEq0bfEcDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzvm_VD3HjU6RMx8zq7g1g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzw0Z6-_VDp9ShIRSKIsQw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzwYhKPhnZQPushMyzgtDg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzz-M4QvkEpUWWPL9RTzLA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q


In [ ]:
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

In [ ]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())